In [1]:
# libraries for general data processing and visualization
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt

# Functional API Model building libraries
import tensorflow as tf
from tensorflow.keras.layers import Dense,GlobalAvgPool2D,Dropout,BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam,SGD
from tensorflow.keras.regularizers import L1
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping,ModelCheckpoint

# importing Libraries of Pre-Trained models for Feature Extration
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.applications.mobilenet import preprocess_input

# libraies for image Augmentation
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
base_model=MobileNet(weights='imagenet',# weights of network 
                     include_top=False, # Boolean, whether to include the fully-connected layer at the top of the network. Defaults to True.
                     input_shape=(224,224,3  )# input shape/size for the model. it allow only standared sizes only
                    )


# other options
# VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
# ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
# InceptionV3(weights='imagenet', include_top=False, input_shape=(299, 299, 3))
# Xception(weights='imagenet', include_top=False, input_shape=(299, 299, 3))

In [3]:
base_model.summary()

Model: "mobilenet_1.00_224"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv1 (Conv2D)              (None, 112, 112, 32)      864       
                                                                 
 conv1_bn (BatchNormalizati  (None, 112, 112, 32)      128       
 on)                                                             
                                                                 
 conv1_relu (ReLU)           (None, 112, 112, 32)      0         
                                                                 
 conv_dw_1 (DepthwiseConv2D  (None, 112, 112, 32)      288       
 )                                                               
                                                                 
 conv_dw_1_bn (BatchNormali  (None, 112, 112, 32

 )                                                               
                                                                 
 conv_dw_8_bn (BatchNormali  (None, 14, 14, 512)       2048      
 zation)                                                         
                                                                 
 conv_dw_8_relu (ReLU)       (None, 14, 14, 512)       0         
                                                                 
 conv_pw_8 (Conv2D)          (None, 14, 14, 512)       262144    
                                                                 
 conv_pw_8_bn (BatchNormali  (None, 14, 14, 512)       2048      
 zation)                                                         
                                                                 
 conv_pw_8_relu (ReLU)       (None, 14, 14, 512)       0         
                                                                 
 conv_dw_9 (DepthwiseConv2D  (None, 14, 14, 512)       4608      
 )        

### Building Model on top of pre-trained model using Functional API: Mobilenet
- imported only feature extration(Convolution layers part) part from pre-trained model: **mobilenet**

- Pneumonia Detection
- classes: Normal and Pneumonia

In [6]:
# for cat and dog problem no of classes are two
classes = 2

# base model output size: (--,7,7,1024)
x=base_model.output 
# global avg pooling size: 1024
x=GlobalAvgPool2D(name='avg_pool')(x)

x=Dense(512,activation='relu',name='FC1')(x)
x=Dropout(0.3,name='dr_FC1')(x) 

# fully connect dense layer with 128 nodes
x=Dense(128,activation='relu',name='FC2')(x)
# batch normalization
#x=BatchNormalization(name='bn_FC1')(x)
# adding dropout layer to counter overfitting
x=Dropout(0.3,name='dr_FC2')(x) 

# fully connected output dense layer for predictions
predictions=Dense(classes,activation='softmax',name='output')(x)

# creating model with input= base model inputs only and outputs= FC output layer 
model=Model(inputs=base_model.input,
            outputs=predictions)

In [7]:
# no of layers in base model
len(base_model.layers)

86

In [8]:
# we are useing pretrained weights in based model
# trainable parameters as False i.e we are not updating trainable parameters in base model
for layer in base_model.layers:
    layer.trainable=False

#pt=SGD(learning_rate=0.01,momentum=0.9)
# compiling the model
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy']) 

In [9]:
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv1 (Conv2D)              (None, 112, 112, 32)      864       
                                                                 
 conv1_bn (BatchNormalizati  (None, 112, 112, 32)      128       
 on)                                                             
                                                                 
 conv1_relu (ReLU)           (None, 112, 112, 32)      0         
                                                                 
 conv_dw_1 (DepthwiseConv2D  (None, 112, 112, 32)      288       
 )                                                               
                                                                 
 conv_dw_1_bn (BatchNormali  (None, 112, 112, 32)      128 

 )                                                               
                                                                 
 conv_dw_8_bn (BatchNormali  (None, 14, 14, 512)       2048      
 zation)                                                         
                                                                 
 conv_dw_8_relu (ReLU)       (None, 14, 14, 512)       0         
                                                                 
 conv_pw_8 (Conv2D)          (None, 14, 14, 512)       262144    
                                                                 
 conv_pw_8_bn (BatchNormali  (None, 14, 14, 512)       2048      
 zation)                                                         
                                                                 
 conv_pw_8_relu (ReLU)       (None, 14, 14, 512)       0         
                                                                 
 conv_dw_9 (DepthwiseConv2D  (None, 14, 14, 512)       4608      
 )        

In [10]:
# data loading fold should have only train and test folders only.
# checking for file names in the directory
os.listdir("F:/YNaiduBabu/DL_by_LalithSachan/Download_Data/chest_xray")

['test', 'train', 'val']

In [11]:
#os.remove("F:/YNaiduBabu/DL_by_LalithSachan/Download_Data/cat_dog_sub/.DS_Store" ) 

In [12]:
#os.listdir("F:/YNaiduBabu/DL_by_LalithSachan/Download_Data/cat_dog_sub") 

### Creating Data Generator:
Since the dataset is big and to avoid memory insufficiency we need to train the model into batches, to achieve this purpose we will use a data generator.

Apart from this, we need to apply data augmentation to avoid overfitting problems.

In the data augmentation, by applying some small transformations we achieve more generalized results. this also leads to generating more training data by applying transformations on it.

The data Image data generator handles all the image processing tasks.

In [13]:
width=224
height=224
batch_size=32
train_dir=r'F:/YNaiduBabu/DL_by_LalithSachan/Download_Data/chest_xray/train'
test_dir=r'F:/YNaiduBabu/DL_by_LalithSachan/Download_Data/chest_xray/test'

# data preparation
# Generate batches of tensor image data with real-time data augmentation.
train_data_gen=ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=20,
    #rescale=1./255,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

validation_data_gen=ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=20, 
    #rescale=1./255,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

### Loading the data into DataGenerator 
The method ***"flow_from_directory"*** loads the data recursively by going to the directory by directory if our main directory is in a hierarchical fashion.

In [14]:
# generating data and passing as batches with specific trget size
train_generator=train_data_gen.flow_from_directory(
    train_dir,
    target_size=(height,width),
    #color_mode='grayscale',# it genrates image of (height, width,3) because colour image: 'rgb'
    batch_size=batch_size,
    class_mode='categorical')

validation_generator=validation_data_gen.flow_from_directory(
    test_dir,
    target_size=(height,width),
    #color_mode='grayscale',
    batch_size=batch_size,
    class_mode='categorical') 

Found 5216 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


### class weights caliculation:

In [ ]:
from sklearn.utils import class_weight

In [ ]:
cw = class_weight.compute_class_weight(class_weight='balanced', 
                                       classes=np.unique(train_generator.classes),
                                       y=train_generator.classes)

cw_dict = dict(enumerate(cw))

In [ ]:
cw_dict

### rough work:

In [ ]:
data=next(train_generator)

In [ ]:
data[0].shape,data[1].shape

In [ ]:
train_generator.n,train_generator.batch_size

In [ ]:
validation_generator.n,validation_generator.batch_size

### Training

In [15]:
EPOCHS =100
STEPS_PER_EPOCH= train_generator.n//train_generator.batch_size
VALIDATION_STEPS=validation_generator.n//train_generator.batch_size

In [16]:
STEPS_PER_EPOCH, VALIDATION_STEPS

(163, 19)

In [17]:
early_stoping=EarlyStopping(monitor='val_loss',
              patience=10)

In [19]:
# creating new folder if it doesnot exists
outputFolder='./chest_xray_model_output'
if not os.path.exists(outputFolder):
  os.makedirs(outputFolder)

In [20]:
# ModelCheckpoint: Callback to save the Keras model or model weights at some frequency.
file_path=outputFolder+'/weights-{epoch:02d}-{loss:.4f}-{accuracy:.4f}-{val_accuracy:.4f}.h5'
checkpoint=ModelCheckpoint(filepath=file_path,
                           save_weights_only=True,
                           monitor='val_accurary',
                           mode='max',
                           save_best_only=False,
                           #save_freq=41
                          )

In [ ]:
# learning_rate_reduction = ReduceLROnPlateau(monitor='val_accuracy', 
#                                             patience = 2, verbose=1,
#                                             factor=0.3, 
#                                             min_lr=0.000001)

In [21]:
model.fit(
    train_generator,
    #class_weight=cw_dict,
    epochs=EPOCHS,
    steps_per_epoch=STEPS_PER_EPOCH,
    validation_data=validation_generator,
    validation_steps=VALIDATION_STEPS, 
    callbacks=[early_stoping,checkpoint],
    verbose=1) 

Epoch 1/100


163/163 [==============================] - 164s 996ms/step - loss: 0.3954 - accuracy: 0.8568 - val_loss: 0.7694 - val_accuracy: 0.7615
Epoch 2/100
163/163 [==============================] - 135s 830ms/step - loss: 0.2000 - accuracy: 0.9214 - val_loss: 1.0169 - val_accuracy: 0.6595
Epoch 3/100
163/163 [==============================] - 136s 834ms/step - loss: 0.1666 - accuracy: 0.9352 - val_loss: 0.2618 - val_accuracy: 0.8980
Epoch 4/100
163/163 [==============================] - 133s 813ms/step - loss: 0.1556 - accuracy: 0.9427 - val_loss: 0.5328 - val_accuracy: 0.8503
Epoch 5/100
163/163 [==============================] - 136s 834ms/step - loss: 0.1469 - accuracy: 0.9461 - val_loss: 0.4308 - val_accuracy: 0.8635
Epoch 6/100
163/163 [==============================] - 132s 811ms/step - loss: 0.1391 - accuracy: 0.9448 - val_loss: 0.3564 - val_accuracy: 0.8651
Epoch 7/100
163/163 [==============================] - 133s 815ms/step - loss: 0.1209 - accuracy: 0.9553 - val_loss:

### Testing 

In [ ]:
# test image directory
myim=r'F:/YNaiduBabu/DL_by_LalithSachan/Download_Data/cat_dog_sub/test/cat/3385.jpg'

# loading the image using preprocessing library: image
img = image.load_img(myim, target_size=(224,224)) 

In [ ]:
img 

In [ ]:
# test image
plt.imshow(img)

In [ ]:
# converting image into  array
x = image.img_to_array(img)
x.shape

In [ ]:
# preprocessing the input simlar to preprocessing of mobilenet input

x = preprocess_input(x)
x.shape 

In [ ]:
# input_1 (InputLayer)        [(None, 224, 224, 3)]. its 4D array
# 3D array into 4D array. as model expects 4D array
x = np.expand_dims(x, axis=0) # expanding dimentions at axis: 0(first)

In [ ]:
x.shape

In [ ]:
# predicting the output from the trained model
# accesig the only probabilities only i.e. model.predict(x)[0]
pred = model.predict(x)[0]
pred 

In [ ]:
# classes for the predicted probabilities
train_generator.class_indices

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image

# Load the image using PIL (Python Imaging Library)

im = Image.open(myim)

# Display the image using matplotlib
plt.imshow(im)
plt.axis('off')
plt.show()